# Working with AutoGen Agents

This lesson explores how to work with AutoGen's agent system, focusing on creating and using different types of agents for various tasks.

## Setup

In [ ]:
import os
import getpass
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Creating a Basic Assistant Agent

Let's create a simple assistant agent that can analyze data and provide insights:

In [ ]:
# Create a custom tool for data analysis
async def analyze_numbers(numbers: str) -> str:
    """Analyze a list of numbers and return basic statistics"""
    nums = [float(n) for n in numbers.split(',')]
    mean = sum(nums) / len(nums)
    maximum = max(nums)
    minimum = min(nums)
    return f"Analysis Results:\nMean: {mean:.2f}\nMax: {maximum}\nMin: {minimum}"

# Initialize the OpenAI client
model_client = OpenAIChatCompletionClient(
    model="gpt-4",
)

# Create the assistant agent
data_analyst = AssistantAgent(
    name="data_analyst",
    model_client=model_client,
    tools=[analyze_numbers],
    system_message="You are a data analysis assistant. Use the analyze_numbers tool to help users understand their numerical data."
)

## Using the Assistant Agent

Let's demonstrate how to interact with the agent:

In [ ]:
async def run_analysis():
    # Create a message asking for analysis
    message = TextMessage(
        content="Can you analyze these numbers: 10,15,20,25,30",
        source="user"
    )
    
    # Get the response using streaming
    await Console(
        data_analyst.on_messages_stream(
            [message],
            cancellation_token=CancellationToken()
        )
    )

# Run the analysis
await run_analysis()

## Creating an Agent with Context Management

Here's how to create an agent with limited context memory:

In [ ]:
from autogen_core.model_context import BufferedChatCompletionContext

# Create an agent with limited context memory
memory_efficient_agent = AssistantAgent(
    name="efficient_assistant",
    model_client=model_client,
    tools=[analyze_numbers],
    system_message="You are a helpful assistant with limited memory.",
    model_context=BufferedChatCompletionContext(buffer_size=3)  # Only remember last 3 messages
)

async def test_memory_agent():
    messages = [
        TextMessage(content="What is 2+2?", source="user"),
        TextMessage(content="Can you multiply 5 by 6?", source="user"),
        TextMessage(content="What was my first question?", source="user")
    ]
    
    for msg in messages:
        await Console(
            memory_efficient_agent.on_messages_stream(
                [msg],
                cancellation_token=CancellationToken()
            )
        )

await test_memory_agent()

## Working with Tools and External Data

Let's create an agent that can work with pandas dataframes:

In [ ]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

# Create a sample dataset
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'age': [25, 30, 35, 40],
    'salary': [50000, 60000, 75000, 80000]
}
df = pd.DataFrame(data)

# Create a tool for pandas operations
pandas_tool = LangChainToolAdapter(
    PythonAstREPLTool(locals={"df": df})
)

# Create an agent that can analyze the dataframe
data_agent = AssistantAgent(
    name="data_analyst",
    tools=[pandas_tool],
    model_client=model_client,
    system_message="You are a data analyst. Use pandas to analyze the dataframe 'df'."
)

async def analyze_dataframe():
    questions = [
        "What is the average age in the dataset?",
        "Who has the highest salary?",
        "Calculate the total salary for all employees."
    ]
    
    for question in questions:
        await Console(
            data_agent.on_messages_stream(
                [TextMessage(content=question, source="user")],
                cancellation_token=CancellationToken()
            )
        )

await analyze_dataframe()

This lesson demonstrates the key concepts of working with AutoGen agents, including:
- Basic agent creation and interaction
- Using tools and external functions
- Managing context and memory
- Working with data analysis capabilities

The examples show how to create specialized agents for different purposes while maintaining a clean and efficient implementation.